# FBI background checks - States and categories

### Import Python tools and Jupyter configuration

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import altair as alt
import datetime as dt
import numpy as np

In [3]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

In [4]:
month_year_updated = dt.date.today().strftime("%m_%Y")

---

### Bless you @jsvine

In [5]:
url = "https://raw.githubusercontent.com/BuzzFeedNews/nics-firearm-background-checks/master/data/nics-firearm-background-checks.csv"

### Read data

In [20]:
src = pd.read_csv(url)

In [27]:
src["year"] = src["month"].str[:4]

### Exclude permits figures

In [28]:
src["totals_no_permits"] = src["totals"] - src["permit"]

In [29]:
df = src[["month", "year", "state", "handgun", "long_gun", "totals_no_permits"]]

In [30]:
df.tail()

,month,year,state,handgun,long_gun,totals_no_permits
15230,1998-11,1998,Virginia,14.0,2.0,24.0
15231,1998-11,1998,Washington,65.0,286.0,360.0
15232,1998-11,1998,West Virginia,149.0,251.0,405.0
15233,1998-11,1998,Wisconsin,25.0,214.0,241.0
15234,1998-11,1998,Wyoming,45.0,49.0,99.0


In [41]:
grouped = (
    df[df["year"] > "2008"]
    .groupby(["year"])
    .agg({"handgun": sum, "long_gun": sum, "totals_no_permits": sum})
    .reset_index()
)

In [42]:
grouped.to_csv("data/processed/total_by_year.csv", index=False)

In [39]:
grouped_states = (
    df[df["year"] > "2008"]
    .groupby(["year", "state"])
    .agg({"handgun": sum, "long_gun": sum, "totals_no_permits": sum})
    .reset_index()
)

In [40]:
grouped_states

,year,state,handgun,long_gun,totals_no_permits
0,2009,Alabama,119867.0,146978.0,315249.0
1,2009,Alaska,23339.0,31112.0,60780.0
2,2009,Arizona,82347.0,71188.0,174236.0
3,2009,Arkansas,55239.0,76494.0,166257.0
4,2009,California,240795.0,256203.0,497049.0
5,2009,Colorado,126647.0,132671.0,313640.0
6,2009,Connecticut,46464.0,40850.0,88344.0
7,2009,Delaware,9712.0,10246.0,20530.0
8,2009,District of Columbia,356.0,2.0,369.0
9,2009,Florida,323436.0,195554.0,556540.0
